Job Analyzer AI

Goal:
A user pastes a job description (or a URL to one), and this app:

-Extracts skills and requirements

-Tells them what they match / lack

-Suggests what to learn

-Generates tailored resume bullets or a cover letter


In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input","nav", "footer", "form", "aside"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
      #   links = [link.get('href') for link in soup.find_all('a')]
      #   self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
job_system_prompt = (
    "You are a helpful assistant that analyzes job descriptions for software developers.\n"
    "Given a job description, extract:\n"
    "- A brief summary of the role\n"
    "- Required skills\n"
    "- Preferred (optional) skills\n"
    "- A list of matched and missing skills (based on user skills provided)\n"
    "- Resume bullet point suggestions\n"
    "- A short custom paragraph for a cover letter\n"
    "Respond in JSON format with those keys: summary, required_skills, preferred_skills, matched_skills, missing_skills, resume_bullets, cover_letter."
)
